In [52]:
import os
import uuid
from pathlib import Path
import glob
import shutil
import json
import numpy as np
import pathlib
import pandas as pd
from collections import namedtuple
import subprocess

from xml.dom import minidom

import networkx as nx
import graphical_models
import tqdm

Make sure  https://github.com/linlinchn/gnw/raw/master/gnw-3.1b.jar is in your root directory (should be there already)

Credits: Code copied from https://github.com/yannadani/cbed/blob/main/envs/dream4.py and adjusted for modularity

Usage:
Different graphs are available in /configurations, just switch them out by setting them in the 'name' argument of {observe, intervene, knockdown}

Note: Knockdown/Knockout only generate one sample a time for the node specified (a limitation by the original Java jar code, can only be improved by adjusting that Java code and recompiling which sounds like pain)


In [56]:
def observe(samples, name='InSilicoSize10-Ecoli1'):
    network = f'configurations/{name}.xml'
    cmd = f'java -jar gnw-3.1b.jar -c settings.txt --input-net {network} --output-net-format=1 --simulate'
    print(cmd)

    data = []
    for _ in tqdm.tqdm(range(samples)):
        subprocess.check_call(cmd.split(' '),  stderr=subprocess.DEVNULL)
        data.append(pd.read_csv(f'{name}_wildtype.tsv', sep='\t'))
    return pd.concat(data).to_numpy()


def intervene(node, name='InSilicoSize10-Ecoli1'):
    network = f'configurations/{name}.xml'
    cmd = f'java -jar gnw-3.1b.jar -c settings.txt --input-net {network} --output-net-format=1 --simulate'
    print(cmd)
    subprocess.check_call(cmd.split(' '),  stderr=subprocess.DEVNULL)
    data = pd.read_csv(f'{name}_knockouts.tsv', sep='\t')
    return data.iloc[node].to_numpy()

def knockdown(node, name='InSilicoSize10-Ecoli1'):
    network = f'configurations/{name}.xml'
    cmd = f'java -jar gnw-3.1b.jar -c settings.txt --input-net {network} --output-net-format=1 --simulate'
    print(cmd)
    subprocess.check_call(cmd.split(' '), stderr=subprocess.DEVNULL)
    data = pd.read_csv(f'{name}_knockdowns.tsv', sep='\t')
    return data.iloc[node].to_numpy()


def get_network(xml):
    xmldoc = minidom.parse(str(xml))

    nodes = []
    var2id = {}
    for i, node in enumerate(xmldoc.getElementsByTagName('species')):
        name = node.attributes.get('id').value
        if 'void' not in name:
            nodes.append(name)
            var2id[name] = i

    A = np.zeros((len(nodes), len(nodes)))

    for node in xmldoc.getElementsByTagName('reaction'):
        # child
        child = node.getElementsByTagName('listOfProducts')[0].getElementsByTagName('speciesReference')[0].attributes.get('species').value

        #parents
        for parent in node.getElementsByTagName('modifierSpeciesReference'):
            _from = var2id[parent.attributes.get('species').value]
            _to = var2id[child]
            A[_from, _to] = 1

    return nodes, var2id, A

In [54]:
observe(samples=5)

java -jar gnw-3.1b.jar -c settings.txt --input-net configurations/InSilicoSize10-Ecoli1.xml --output-net-format=1 --simulate


100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


array([[0.0640397, 1.0130489, 0.050506 , 0.1461571, 0.9247111, 0.1156813,
        0.3843457, 1.0536961, 0.9902394, 1.096382 ],
       [0.059421 , 1.0065962, 0.0447414, 0.1496074, 0.9452848, 0.0686235,
        0.4525003, 1.0334531, 1.0199004, 0.973445 ],
       [0.0721182, 1.0754532, 0.0754594, 0.1197127, 0.9472995, 0.1265829,
        0.3578588, 1.0552794, 1.0229341, 1.0139334],
       [0.0678927, 0.9973144, 0.0526402, 0.1167154, 0.9720499, 0.0795689,
        0.4154059, 0.9945208, 1.0193882, 1.046124 ],
       [0.0686383, 1.1270065, 0.0561479, 0.1814998, 0.9480983, 0.314055 ,
        0.3127178, 1.0578455, 0.9703489, 1.0934002]])

In [60]:
nodes, var2id, A= get_network('InSilicoSize10-Ecoli1.xml')
nodes, var2id, A

(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10'],
 {'G1': 0,
  'G2': 1,
  'G3': 2,
  'G4': 3,
  'G5': 4,
  'G6': 5,
  'G7': 6,
  'G8': 7,
  'G9': 8,
  'G10': 9},
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]))

In [61]:
import matplotlib.pyplot as plt
import networkx as nx

def show_graph_with_labels(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.DiGraph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, labels=mylabels, with_labels=True, arrows=True)
    plt.show()
show_graph_with_labels(A, {i:n for i, n in enumerate(nodes)})

In [62]:
intervene(node=1) #only one sample at a time

java -jar gnw-3.1b.jar -c settings.txt --input-net configurations/InSilicoSize10-Ecoli1.xml --output-net-format=1 --simulate


array([0.9999905, 0.       , 1.0266926, 0.3496831, 0.8092653, 1.048231 ,
       0.0733952, 1.1125337, 1.0573059, 1.0577539])

In [63]:
knockdown(node=1) #only one sample at a time

java -jar gnw-3.1b.jar -c settings.txt --input-net configurations/InSilicoSize10-Ecoli1.xml --output-net-format=1 --simulate


array([0.1147207, 0.439091 , 0.1640415, 0.1978809, 0.8013127, 0.9053036,
       0.3581723, 1.0286293, 1.0297592, 0.933466 ])